## Kafka Library
실행을 위해 다음 패키지 설치가 필요합니다.

```
pyenv activate pyspark 

pip install  kafka-python==2.0.2 msgpack==1.0.3 pandas==1.3.4
```

In [1]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.client_async import KafkaClient
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import TopicPartition

import json
import msgpack

### Kafka Broker 정보 확인

Docker Compose 환경을 가정합니다. (로컬 환경 내 Kafka Broker 및 Zookeeper 실행)

In [2]:
zookeeper_servers = ["127.0.0.1:2181"]
bootstrap_servers = ["127.0.0.1:9092"]

### Kafka Python API 사용

[kafka-python](https://kafka-python.readthedocs.io/en/master/index.html) API 를 사용합니다.  
[confluent-kafka-python](https://github.com/confluentinc/confluent-kafka-python) 을 사용할 수도 있습니다.

In [3]:
client = KafkaClient(bootstrap_servers=bootstrap_servers)
client.cluster.brokers()

{BrokerMetadata(nodeId='bootstrap-0', host='127.0.0.1', port=9092, rack=None)}

In [4]:
admin = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

In [5]:
admin.list_consumer_groups()

[('schema-registry', 'sr'),
 ('user-event-relay-5c23dcf9-45d8-4493-9527-bc0316729aeb--409846158-driver-0',
  'consumer')]

### Topic 생성

Kafka Partition 을 생성합니다. 일반적으로는 Kafka 관리자가 요청을 받아 UI 나 Console 에서 직접 생성할 수 있습니다.  
API 를 이용해 생성하는 경우는 많지 않으나, 이 노트북에선 실습을 위해 API 를 통해 생성하는 예제를 사용합니다.

In [ ]:
# replication = 1, partition = 1 의 "airbnb_listing_test" 라는 토픽을 생성합니다.
topic = "user-event"
admin.create_topics([NewTopic(topic, 1, 1)])

In [12]:
admin.list_topics()

['user-event', '_schemas', '__consumer_offsets']

### Kafka Producer 를 이용해 데이터 전송

kafka-python API 를 이용해 데이터를 Broker 로 전송합니다.  
일반적으로는 Java / Scala 언어로 이루어진 개발한 Producer 나 Kafka Connect 등을 이용합니다.  

다만 이 노트북에선 실습으로 Python API 를 이용하기 위해 Pandas 로 CSV 파일을 읽어 Python Kafka API 로 전송합니다.

In [6]:
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [7]:
DATASET_ROOT = "/Users/kun/github/1ambda/practical-data-pipeline-code"
DATASET_AIRBNB_LISTING = f"{DATASET_ROOT}/_datasets/ecommerce/2020-Feb.csv"

In [8]:
import pandas as pd
pdf = pd.read_csv(DATASET_AIRBNB_LISTING)

In [9]:
pdf.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-02-01 00:00:01 UTC,cart,5844305,1487580006317032337,NaN,NaN,2.14,485174092,4be9643a-420b-4c6b-83dd-a15e772fbf7a
1,2020-02-01 00:00:03 UTC,view,5769925,1487580013841613016,NaN,kapous,4.22,594621622,a88baf11-9cd0-4362-bde4-1bfeed3f641d


In [12]:
# Pandas Row 는 Tuple 입니다. 이를 Dict 로 바꾸기 위해 to_dict 함수를 호출해
# 전달 받은 Row (Dict) 를 Kafka Producer 를 이용해 Topic 으로 보냅니다.

for row in pdf.head(10).to_dict(orient="records"):
    del row['category_id']
    del row['category_code']
    del row['brand']
    message = json.dumps(row).encode('utf-8')

    producer.send(topic, message)

### Console Consumer 로 데이터 읽기

이제 Broker 에 접근해 Console Consumer 로 해당 토픽 데이터를 읽어봅니다. 컨테이너에 들어가 다음 커맨드를 실행할 수 있습니다.  

```
kafka-console-consumer --bootstrap-server broker:9092 --topic airbnb_listing_test --from-beginning
```